<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_1115PyTorch_buildmodel_tutorial_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

# ニューラルネットワークの構築
<!-- # Build the Neural Network -->

ニューラルネットワークは、データに対して演算を行う層やモジュールで構成されています。
[torch.nn](https://pytorch.org/docs/stable/nn.html) 名前空間は、独自のニューラルネットワークを構築するために必要なすべてのビルディングブロックを提供します。
独自のニューラルネットワークを構築するために必要なすべてのビルディングブロックを提供します。
PyTorch の全てのモジュールは [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) をサブクラス化しています。
ニューラルネットワークは、モジュール自体が他のモジュール(層)で構成されています。
この入れ子構造により、複雑なアーキテクチャを簡単に構築・管理することができます。

次節では FashionMNIST データセットの画像を分類するためのニューラルネットワークを構築します。
<!-- 
Neural networks comprise of layers/modules that perform operations on data.
The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to
build your own neural network. 
Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers). 
This nested structure allows for building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.
-->


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 1. 学習用デバイスの入手
<!-- # 1. Get Device for Training -->

利用可能であれば ハードウェアアクセラレータ GPU でモデルを学習できるようにします。
利用可能かどうか確認してみましょう。
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html)が利用可能かどうかを確認します。
そうでなければ，引き続き CPU を使用します。
<!--
We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. Let's check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we continue to use the CPU.
-->


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


# 2. クラスの定義
<!-- # Define the Class -->

ニューラルネットワークを定義するには、`nn.Module`をサブクラス化し、`__init__`でニューラルネットワークの層を初期化します。
すべての `nn.Module` サブクラスは、入力データに対する操作を `forward` メソッドで実装します。
<!--
We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. 
Every `nn.Module` subclass implements the operations on input data in the `forward` method. 
-->



In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

`NeuralNetwork` の実体(インスタンス)を作成し，それを `デバイス` に移動させ，その構造を表示します。
<!-- We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure. -->



In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


モデルを使用するには，入力データを渡します。
これにより，モデルの `forward` が実行されます。
いくつかの [バックグラウンド操作](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866) が実行されます。
`model.forward()` を直接呼び出さない!

入力に対してモデルを呼び出すと，各クラスの生の予測値を持つ 10 次元のテンソルが返されます。
これを `nn.Softmax` モジュールのインスタンスに渡すことで，予測確率を得ることができます．

<!-- To use the model, we pass it the input data. 
This executes the model's `forward`,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.
-->


In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

--------------




# 3. モデルの層
<!-- # 3. Model Layers -->

FashionMNIST モデルの層を分解してみましょう。
それを説明するために サイズ 28x28 の 3 枚の画像のサンプルミニバッチを取り，それをネットワークに通すとどうなるかを見てみましょう。
<!--
Let's break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.
-->


In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## 3.1 `nn.Flatten` (フラット化)
<!-- ## 3.1 `nn.Flatten` -->

[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) 層を初期化して，各 2D 28x28 画像を 784 画素値の連続した配列に変換します (ミニバッチの次元(dim=0) は維持されます)。
<!--
We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).
-->


In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

## 3.2 `nn.Linear` (線形層)
<!-- ## 3.2 `nn.Linear` -->

[線形層 `nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)は、保存されている重みとバイアスを使って、入力に線形変換を施すモジュールです。
<!--
The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.
-->


In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

## 3.3 `nn.ReLU`
<!-- ## 3.3 `nn.ReLU` -->

非線形活性化は、モデルの入力と出力の間の複雑なマッピングを作成するものです。
非線形活性化は、線形変換の後に適用され **非線形性** を導入することで， ニューラルネットワークが様々な現象を学習するのに役立ちます。

このモデルでは、線形層の間に[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) を使用していますが、あなたのモデルに非線形性を導入するための他の活性化もあります。
<!--
Non-linear activations are what create the complex mappings between the model's inputs and outputs. 
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there's other activations to introduce non-linearity in your model.
-->


In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

## 3.4 `nn.Sequential`
<!-- ## `nn.Sequential` -->

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)は、モジュールの順序付きコンテナです。
データは定義されているのと同じ順番ですべてのモジュールに渡されます。
シーケンシャルコンテナを使って、`seq_modules`のような素早いネットワークを組むことができます。
<!--
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. 
The data is passed through all the modules in the same order as defined. 
You can use sequential containers to put together a quick network like `seq_modules`.
-->


In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## 3.5 `nn.Softmax`
<!-- ## `nn.Softmax` -->

ニューラルネットワークの最後の線形層は $[-\infty, \infty]$ の生の値である`logits`を返し，これは[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) モジュールに渡されます。
logits は，各クラスに対するモデルの予測確率を表す値 [0, 1] にスケーリングされます。
`dim`パラメータは， 値の合計が 1 になるような次元を示します。
<!--
The last linear layer of the neural network returns `logits` - raw values in $[-\infty, \infty]$ - which are passed to the [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. 
The logits are scaled to values [0, 1] representing the model's predicted probabilities for each class. 
`dim` parameter indicates the dimension along which the values must sum to 1. 
-->



In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# 4. モデルパラメータ
<!-- # Model Parameters -->

ニューラルネットワークの多くの層は、パラメータ化されています。つまり、学習時に最適化される重みやバイアスが関連付けられています。nn.Module`をサブクラス化すると、モデルオブジェクト内で定義されたすべてのフィールドが自動的に追跡され、モデルの`parameters()`や`named'
モデルの `parameters()` や `named_parameters()` メソッドを使って、すべてのパラメータにアクセスできるようになります。

この例では，各パラメータを繰り返し処理し，そのサイズと値のプレビューを表示しています．
<!--
Many layers inside a neural network are *parameterized*, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters
accessible using your model's `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.
-->

In [ ]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

--------------




# Further Reading

- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

